In [1]:
import numpy as np
import timeit
np.random.seed(1000)

In [2]:
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N,1)

In [3]:
def evalf_l(x, lambd):
  res = np.multiply(lambd/2,np.matmul(x.transpose(),x))+((np.linalg.norm(np.subtract(np.matmul(A,x),y)))**2)/2
  return res

In [4]:
def evalg_l(x, lambd):
  n = x.shape[0]
  arr = np.zeros(n)
  err = np.subtract(np.matmul(A,x),y)
  for i in range(n):
    arr[i] = lambd*x[i] + np.matmul(err.transpose(),A[:,i])
  return arr.reshape(n,1)

In [ ]:
def evalh_l(x, lambd):
  n = x.shape[0]
  hes = np.zeros((n,n))
  for i in range(n):
    for j in range(n):
      hes[i][j] = np.matmul(A[:,i],A[:,j])
      if i == j:
        hes[i][j] += lambd
  return hes

In [ ]:
def compute_steplength_backtracking_scaled_direction(x, p, gradf, alpha_start, rho, gamma, d_k, lambd):
  #assert type(x) is np.ndarray and len(x) == 2
  #assert type(gradf) is np.ndarray and len(gradf) == 2
  
  alpha = alpha_start

  while evalf_l(x+alpha*np.matmul(d_k,p), lambd) > evalf_l(x, lambd) + gamma*alpha*(np.matmul(gradf.transpose(),np.matmul(d_k,p))):
    alpha = rho*alpha

  return alpha

In [ ]:
def Newton_l(n, tol, lambd, *args):
  xlist = []
  x = np.zeros(n).reshape(n,1)
  xlist.append(x)
  grad_f = evalg_l(x, lambd)

  alpha = args[0]
  rho = args[1]
  gamma = args[2]

  hes_f = evalh_l(x, lambd)

  k = 0

  while np.linalg.norm(grad_f) > tol and k < 10000:
    p = -grad_f
    d = np.linalg.inv(hes_f)
    step_length = compute_steplength_backtracking_scaled_direction(x, p, grad_f, alpha, rho, gamma, d, lambd)
    x = np.add(x, np.multiply(step_length, np.matmul(d,p)))
    xlist.append(x)
    k += 1 
    grad_f = evalg_l(x, lambd) 
    hes_f = evalh_l(x, lambd)
  return x

In [ ]:
for i in range(np.size(ds)):
  d=ds[i]
  A = np.random.randn(N,d)
  for j in range(A.shape[1]):
    A[:,j] = A[:,j]/np.linalg.norm(A[:,j])
    
  xorig = np.ones((d,1))
  y = np.dot(A,xorig) + eps

  start = timeit.default_timer()
  x_opt = Newton_l(d, 1e-5, lambda_reg, 1, 0.5, 0.5)
  newtontime = timeit.default_timer() - start
  print('For',d,': Time taken :',newtontime,'\n||Ax* - y||^2 =',(np.linalg.norm(np.subtract(np.matmul(A,x_opt),y)))**2,'\n||x*-xorig||^2 =',(np.linalg.norm(np.subtract(x_opt,xorig)))**2,'\n')

For 1000 : Time taken : 9.93499467200013 
||Ax* - y||^2 = 6.0108776056799973e-05 
||x*-xorig||^2 = 875.8122987367859 

For 5000 : Time taken : 139.44050470799993 
||Ax* - y||^2 = 8.63961256789895e-06 
||x*-xorig||^2 = 4817.355553683237 

For 10000 : Time taken : 599.9326577649999 
||Ax* - y||^2 = 4.05446978527579e-06 
||x*-xorig||^2 = 9815.561161563599 



#Ans 1:

For Newton's method:

For 1000 : Time taken : 9.93499467200013 

$||Ax^* - y||^2$ = 6.0108776056799973e-05 

$||x^*-x_{orig}||^2$ = 875.8122987367859 


For 5000 : Time taken : 139.44050470799993 

$||Ax^* - y||^2$ = 8.63961256789895e-06 

$||x^*-x_{orig}||^2$ = 4817.355553683237 


For 10000 : Time taken : 599.9326577649999 

$||Ax^* - y||^2$ = 4.05446978527579e-06 

$||x^*-x_{orig}||^2$ = 9815.561161563599

For 20000, the runtime exceeds 20 minutes, so we declare it as a failure dimension.


In [ ]:
def compute_steplength_backtracking(x, gradf, B, alpha_start, rho, gamma, lambd):
  #assert type(x) is np.ndarray and len(x) == 2
  #assert type(gradf) is np.ndarray and len(gradf) == 2
  
  alpha = alpha_start

  while evalf_l(x+alpha*-np.matmul(B,gradf), lambd) > evalf_l(x, lambd) + gamma*alpha*np.matmul(gradf.transpose(),-np.matmul(B,gradf)):
    alpha = rho*alpha

  return alpha

In [ ]:
def BFGS_l(n, tol, lambd, *args):
  xlist = []
  x1 = np.zeros(n).reshape(n,1)
  xlist.append(x1)
  grad_f = evalg_l(x1, lambd)

  alpha_start = args[0]
  rho = args[1]
  gamma = args[2]

  I = np.identity(n)

  B = I

  k = 0

  while np.linalg.norm(grad_f) > tol and k < 3000:
    alpha = compute_steplength_backtracking(x1, grad_f, B, alpha_start, rho, gamma, lambd)
    x2 = np.add(x1, np.multiply(alpha,np.matmul(B,-grad_f)))
    s = x2 - x1
    y = evalg_l(x2, lambd) - evalg_l(x1, lambd)
    mu = 1/np.matmul(y.transpose(),s)
    #print(p,alpha,x2,s,y,mu)
    B = np.add(np.matmul(np.matmul(np.subtract(I,np.matmul(np.multiply(mu,s),y.transpose())),B),np.subtract(I,np.matmul(np.multiply(mu,y),s.transpose()))),np.matmul(np.multiply(mu,s),s.transpose()))
    x1 = x2
    xlist.append(x1)
    grad_f = evalg_l(x2, lambd)
    k = k+1
    #print(np.linalg.norm(grad_f))
  return x1

In [ ]:
for i in range(np.size(ds)):
  d=ds[i]
  A = np.random.randn(N,d)
  for j in range(A.shape[1]):
    A[:,j] = A[:,j]/np.linalg.norm(A[:,j])
    
  xorig = np.ones((d,1))
  y = np.dot(A,xorig) + eps

  start = timeit.default_timer()
  x_opt = BFGS_l(d, 1e-5, lambda_reg, 1, 0.5, 0.5)
  bfgstime = timeit.default_timer() - start
  print('For',d,': Time taken :',bfgstime,'\n||Ax* - y||^2 =',(np.linalg.norm(np.subtract(np.matmul(A,x_opt),y)))**2,'\n||x*-xorig||^2 =',(np.linalg.norm(np.subtract(x_opt,xorig)))**2,'\n')

For 1000 : Time taken : 9.922479796999994 
||Ax* - y||^2 = 5.672486123595797e-05 
||x*-xorig||^2 = 865.3153037913714 

For 5000 : Time taken : 686.8632253779999 
||Ax* - y||^2 = 9.791829840368583e-06 
||x*-xorig||^2 = 4783.681693162326 



#Ans 1:

For BFGS method:

For 1000 : Time taken : 9.922479796999994 

$||Ax^* - y||^2$ = 5.672486123595797e-05 

$||x^*-x_{orig}||^2$ = 865.3153037913714 


For 5000 : Time taken : 686.8632253779999 

$||Ax^* - y||^2$ = 9.791829840368583e-06 

$||x^*-x_{orig}||^2$ = 4783.681693162326 

For 10000, the runtime exceeds 20 minutes, so we declare it as a failure dimension.

#Ans 2:

Failure dimension for Newton's method: 20000

Failure dimension for BFGS method: 10000